<a href="https://colab.research.google.com/github/Vaish181204/Suvidha-Foundation/blob/main/Llama_3_8B001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U bitsandbytes accelerate transformers peft evaluate rouge_score bert-score

In [ ]:
# Installing Unsloth and essential evaluation tools
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install -q -U evaluate rouge_score bert-score nltk

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-hz59z35p/unsloth_1fd8c0c2a0634a7f990ec21a99b269fa
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-hz59z35p/unsloth_1fd8c0c2a0634a7f990ec21a99b269fa
  Resolved https://github.com/unslothai/unsloth.git to commit a50b4337ae526744711f54f9aed6e82c778280fd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of unsloth[colab-new] to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following versions that require a different python version: 2025.3.2 Requires-Python <=3.12,>=3.9
ERROR: Could not find a version that satisfies the requirement unsloth_zoo>=2026.2.1; extra == "colab-new" (from unsloth[colab-new]) (from versions: 2024.10.0, 2024.10.1, 2024.10.2, 2024.10.3, 2024.10.4, 20

In [ ]:
# 1. Preferred installation method to avoid colab-new errors
!pip install unsloth
# 2. Force install dependencies to ensure compatibility
!pip install --no-deps xformers trl peft accelerate bitsandbytes
# 3. Required for metrics
!pip install -q -U evaluate rouge_score bert-score nltk

  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
  Using cached huggingface_hub-0.36.2-py3-none-any.whl.metadata (15 kB)
Using cached transformers-4.57.6-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.36.2-py3-none-any.whl (566 kB)
Using cached trl-0.24.0-py3-none-any.whl (423 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.4.1
    Uninstalling huggingface_hub-1.4.1:
      Successfully uninstalled huggingface_hub-1.4.1
  Attempting uninstall: transformers
    Found existing installation: transformers 5.1.0
    Uninstalling transformers-5.1.0:
      Successfully uninstalled transformers-5.1.0
  Attempting uninstall: trl
    Found existing installation: trl 0.14.0
    Uninstalling trl-0.14.0:
      Successfully uninstalled trl-0.14.0


In [ ]:
from unsloth import FastLanguageModel
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# FIXED: Removed task_type to avoid the 'multiple values' TypeError
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
)
print("SUCCESS: Llama-3-8B adapters attached!")

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 5.1.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from unsloth import FastLanguageModel
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

# 1. Load with strict memory limits
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,   # Keep at 2048 to prevent VRAM spikes
    dtype = None,            # Auto-detects Float16 for T4
    load_in_4bit = True,
)

# 2. Add LoRA Adapters (Removed task_type to prevent previous TypeError)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
)
print("SUCCESS: Llama-3-8B loaded on T4 GPU!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.1.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


SUCCESS: Llama-3-8B loaded on T4 GPU!


In [ ]:
import pandas as pd
from datasets import Dataset

# 1. Load and Clean
df = pd.read_excel("/content/NewsSumm_Final_Cleaned.xlsx")
df = df.map(lambda x: str(x) if pd.notnull(x) else x)
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1, seed=42)

# 2. Llama-3-Instruct Prompt Format
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Summarize the article.<|eot_id|><|start_header_id|>user<|end_header_id|>
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{}<|eot_id|>"""

def format_prompts(examples):
    texts = [prompt_template.format(a, s) for a, s in zip(examples["article_text"], examples["human_summary"])]
    return { "text" : texts }

tokenized_dataset = dataset.map(format_prompts, batched = True)
print("SUCCESS: Dataset prepared for Llama-3!")

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

SUCCESS: Dataset prepared for Llama-3!


In [ ]:
# 1. Uninstall the problematic version and install the stable one
!pip uninstall trl -y
!pip install --no-cache-dir --force-reinstall --no-deps "trl<0.15.0"

# 2. Restart the session after installation
# Click Runtime -> Restart session

Found existing installation: trl 0.24.0
Uninstalling trl-0.24.0:
  Successfully uninstalled trl-0.24.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 15.6 MB/s eta 0:00:00


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_dataset["train"],
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 3,          # 3 Epochs for accuracy
        learning_rate = 2e-4,
        fp16 = True,
        optim = "adamw_8bit",
        logging_steps = 10,
        output_dir = "llama3_outputs",
    ),
)

print("Starting Llama-3-8B Training...")
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/900 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting Llama-3-8B Training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 900 | Num Epochs = 3 | Total steps = 339
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


...

wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
10,2.046600
20,1.881400
30,1.900900
40,1.872000
50,1.811800
60,1.822100
70,1.802600
80,1.830500
90,1.946000
100,1.801000


wandb: WARNING URL not available in offline run


train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
train/grad_norm,▃▁▂▁▁▃▂▂▂▂▂▂▃▃▃▄▄▄▅▅▅▄▄▇▆█▇██▇▅██
train/learning_rate,███▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
train/loss,█▇▇▇▆▆▆▆▇▆▆▅▅▄▅▄▄▄▄▅▅▄▃▂▂▁▂▂▁▁▂▂▂
total_flos,8.685125544035942e+16
train/epoch,3
train/global_step,339
train/grad_norm,0.98419
train/learning_rate,1e-05
train/loss,1.2982


TrainOutput(global_step=339, training_loss=1.603506318939119, metrics={'train_runtime': 3671.5377, 'train_samples_per_second': 0.735, 'train_steps_per_second': 0.092, 'total_flos': 8.685125544035942e+16, 'train_loss': 1.603506318939119, 'epoch': 3.0})

In [ ]:
import evaluate
from bert_score import score
rouge = evaluate.load("rouge")

def run_evaluation():
    FastLanguageModel.for_inference(model) # 2x faster inference

    # FIXED: Explicitly convert Column to a standard Python List
    refs = dataset["test"]["human_summary"]
    if not isinstance(refs, list):
        refs = list(refs)

    preds = []
    # FIXED: Ensure this is also a list for iteration
    articles = list(dataset["test"]["article_text"])

    print(f"Generating summaries for {len(articles)} test samples...")
    for article in articles:
        inputs = tokenizer([prompt_template.format(article, "")], return_tensors = "pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens = 128)
        decoded = tokenizer.batch_decode(outputs)[0]

        # Extract assistant response only
        summary = decoded.split("assistant<|end_header_id|>\n")[-1].replace("<|eot_id|>", "").strip()
        preds.append(summary)

    # 1. Calculate ROUGE
    r = rouge.compute(predictions=preds, references=refs)

    # 2. Calculate BERTScore (Now both are lists, so '+' will work internally)
    P, R, F1 = score(preds, refs, lang="en", verbose=False)

    print(f"\nFinal Results for Llama-3-8B:")
    print(f"ROUGE-1: {r['rouge1']:.4f}")
    print(f"ROUGE-2: {r['rouge2']:.4f}")
    print(f"ROUGE-L: {r['rougeL']:.4f}")
    print(f"BERTScore: {F1.mean().item():.4f}")

run_evaluation()

Generating summaries for 100 test samples...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Final Results for Llama-3-8B:
ROUGE-1: 0.4954
ROUGE-2: 0.2682
ROUGE-L: 0.3580
BERTScore: 0.9049
